Нам необходимо сравнить работу двух различных моделей трансформеров в задаче машинного перевода. Для начала зайдем в huggingface, откуда и будем брать претрейновые модели и токенайзеры.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

### Пример 1: Модель Llama.

In [5]:
pip install --no-cache-dir transformers sentencepiece

In [50]:
article_start = "Я люблю мою страну" # Начальное предложение

In [29]:
model = MBartForConditionalGeneration.from_pretrained("SnypzZz/Llama2-13b-Language-translate")
tokenizer = MBart50TokenizerFast.from_pretrained("SnypzZz/Llama2-13b-Language-translate", src_lang="ru_RU")

In [51]:
model_inputs = tokenizer(article_start, return_tensors="pt")

In [52]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
article_en = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_en

['I love мою country']

Как можно заметить, у данной модели есть недостатки: она знает переводы далеко не всех слов. Таким образом, слово "мою" с русского на английский она перевести не может.

In [54]:
tokenizer = MBart50TokenizerFast.from_pretrained("SnypzZz/Llama2-13b-Language-translate", src_lang="en_XX")
model_inputs = tokenizer(article_en, return_tensors="pt")

In [55]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["es_XX"]
)
article_es = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_es

['Yo amo mi país']

Однако при переводе с английского на испанский она смогла перевести "мою"

In [56]:
tokenizer = MBart50TokenizerFast.from_pretrained("SnypzZz/Llama2-13b-Language-translate", src_lang="es_XX")
model_inputs = tokenizer(article_es, return_tensors="pt")

In [57]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"]
)
article_kr = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_kr

['Yo amo mi país']

In [61]:
tokenizer = MBart50TokenizerFast.from_pretrained("SnypzZz/Llama2-13b-Language-translate", src_lang="es_XX")
model_inputs = tokenizer(article_es, return_tensors="pt")

In [62]:
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
)
article_ru = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_ru

['Yo amo mi país']

### Пример 2. Модель Mbart.

In [63]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [69]:
article_start

'Я люблю мою страну'

In [65]:
tokenizer.src_lang = "ru_RU"
model_inputs = tokenizer(article_start, return_tensors="pt")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
article_en2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_en2

['I love my country']

In [67]:
tokenizer.src_lang = "en_XX"
model_inputs = tokenizer(article_en2, return_tensors="pt")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["es_XX"]
)
article_es2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_es2

['A pesar de los esfuerzos realizados por el Gobierno de la República Popular Democrática de Corea y la comunidad internacional, la situación sigue siendo frágil.']

При переводе с испанского на корейский произошла ошибка, перевести модель не смогла. Поэтому попробуем перевод с испанского на близкий к корейскому язык -- японский.

In [71]:
tokenizer.src_lang = "es_XX"
model_inputs = tokenizer(article_es2, return_tensors="pt")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ja_XX"]
)
article_kr2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_kr2

['国際連合(国際連合)は、国際連合(国際連合)は、国際連合(国際連合)は、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連合)が、国際連合(国際連']

Далее переведем с японского на корейский и посмотрим, что из этого выйдет

In [73]:
tokenizer.src_lang = "ja_XX"
model_inputs = tokenizer(article_kr2, return_tensors="pt")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"]
)
article_adadad2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_adadad2

['이란은 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란, 이란']

In [75]:
tokenizer.src_lang = "ko_KR"
model_inputs = tokenizer(article_adadad2, return_tensors="pt")
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
)
article_ru2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
article_ru2

['Iran, Iran, Iran, Iran, Iran, Iran, Iran, Iran, Iran, Iran, Iran, Iran, Iran.']

Как можно заметить, среди двух моделей -- Llama и Mbart выбор модели очевиден: стоит использовать Mbart, так как она лучше показала себя при переводах. Однако у обоих моделей есть очевидные минусы, их нужно дообучать.\
В сравнении двух моделей можно выделить следующие важные моменты:\
Модель Llama, которая была использована в примере 1, не знает некоторых языков, в следствие чего теряется семантика предложения или остутствует перевод вовсе.\
Модель Mbart, которая была использована в примере 2, знает больше языков, однако может во время перевода координально изменить контекст изначального предложения или продублировать его (что, собственно, она и сделала).